In [6]:
from tensorflow.keras.models import load_model
import pickle
import numpy as np

In [7]:
try:
    loaded_model = load_model('asl_model.keras')
except:
    # Fallback if custom objects aren't registered
    loaded_model = load_model('asl_model.keras', compile=False)
    loaded_model.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

# 2. Load the label encoder
with open('label_encoder.pkl', 'rb') as f:
    loaded_le = pickle.load(f)

In [9]:
# 3. Now verify both are loaded correctly
print("=== Model Summary ===")
loaded_model.summary()


=== Model Summary ===


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_39 (Dense)                │ (None, 128)            │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_27          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_31 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_28          │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_32 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_42 (Dense)                │ (None, 36)             │         1,188 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,484 (80.02 KB)

 Trainable params: 20,100 (78.52 KB)

 Non-trainable params: 384 (1.50 KB)

In [10]:

print("\n=== Available ASL Classes ===")
print(list(loaded_le.classes_))


=== Available ASL Classes ===


AttributeError: 'numpy.ndarray' object has no attribute 'classes_'

In [13]:
from tensorflow.keras.models import load_model
import pickle
import numpy as np
from tensorflow.keras.saving import register_keras_serializable

# 1. Register the custom loss function (if needed)
@register_keras_serializable()
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fn(y_true, y_pred):
        import tensorflow as tf
        pt = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
        return -tf.reduce_mean(alpha * tf.pow(1. - pt, gamma) * tf.math.log(pt + 1e-8))
    return focal_loss_fn

# 2. Load model with custom objects
try:
    loaded_model = load_model('asl_model.keras', 
                            custom_objects={'focal_loss_fn': focal_loss()})
except Exception as e:
    print(f"Error loading with custom loss: {e}\nLoading without compilation...")
    loaded_model = load_model('asl_model.keras', compile=False)
    loaded_model.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

# 3. Load and verify labels (handling numpy array case)
with open('label_encoder.pkl', 'rb') as f:
    label_data = pickle.load(f)
    
if isinstance(label_data, np.ndarray):
    class_labels = label_data
    print("\n=== Loaded from numpy array ===")
else:  # Assume it's a LabelEncoder
    class_labels = label_data.classes_
    print("\n=== Loaded from LabelEncoder ===")

print(f"Total classes: {len(class_labels)}")
print("Class labels:", class_labels)

# 4. Prediction function
def predict_asl_sign(landmarks):
    """Enhanced prediction with error handling"""
    try:
        landmarks = np.array(landmarks).reshape(1, -1).astype('float32')
        if landmarks.shape[1] != 63:
            raise ValueError(f"Expected 63 landmarks, got {landmarks.shape[1]}")
            
        preds = loaded_model.predict(landmarks, verbose=0)[0]
        sorted_preds = sorted(zip(class_labels, preds), 
                            key=lambda x: x[1], reverse=True)
        
        return {
            'top': sorted_preds[0],
            'top_3': sorted_preds[:3],
            'all': sorted_preds,
            'is_digit': sorted_preds[0][0] in '0123456789'
        }
    except Exception as e:
        print(f"Prediction error: {e}")
        return None

# Test prediction
sample = np.random.rand(63).tolist()  # Replace with real data
result = predict_asl_sign(sample)

if result:
    print("\nTest Prediction:")
    print(f"Top: {result['top'][0]} ({result['top'][1]:.1%})")
    print("Top 3:")
    for char, prob in result['top_3']:
        print(f"- {char}: {prob:.1%}")


=== Loaded from numpy array ===
Total classes: 36
Class labels: ['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h'
 'i' 'j' 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']

Test Prediction:
Top: 0 (70.5%)
Top 3:
- 0: 70.5%
- q: 9.0%
- o: 6.5%


In [15]:
# Add these right after your existing prediction code

MIN_CONFIDENCE = 0.7
CONFUSION_PAIRS = {
    '0': ['o', 'q'],
    '1': ['l'],
    '5': ['s'],
    '6': ['b'],
    '9': ['g']
}

def check_confusions(prediction):
    char, _ = prediction['top']
    for similar in CONFUSION_PAIRS.get(char, []):
        if similar in [x[0] for x in prediction['top_3']]:
            print(f"Visual similarity warning: {char.upper()} ↔ {similar.upper()}")

def display_prediction(result):
    if not result:
        print("No prediction available")
        return
    
    char, confidence = result['top']
    char = char.upper()  # Convert to uppercase
    
    print(f"\n{'='*40}")
    print(f"Predicted: {char} ({confidence:.1%} confidence)")
    
    if confidence < MIN_CONFIDENCE:
        print("⚠️ Low confidence warning")
    
    print("\nTop alternatives:")
    for i, (alt_char, alt_prob) in enumerate(result['top_3'][1:], 1):
        print(f"{i}. {alt_char.upper()} ({alt_prob:.1%})")
    
    check_confusions(result)
    print('='*40)

# Test it
display_prediction(result)


Predicted: 0 (70.5% confidence)

Top alternatives:
1. Q (9.0%)
2. O (6.5%)
Visual similarity warning: 0 ↔ O
Visual similarity warning: 0 ↔ Q
